# Module 6: Peer Reviewed Assignment

### Outline:
The objectives for this assignment:

1. Apply the processes of model selection with real datasets.
2. Understand why and how some problems are simpler to solve with some forms of model selection, and others are more difficult.
3. Be able to explain the balance between model power and simplicity.
3. Observe the difference between different model selection criterion.

General tips:

1. Read the questions carefully to understand what is being asked.
2. This work will be reviewed by another human, so make sure that you are clear and concise in what your explanations and answers.

In [1]:
# This cell loads in the necesary packages
library(tidyverse)
library(leaps)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Problem 1: We Need Concrete Evidence!

[Ralphie](https://en.wikipedia.org/wiki/Ralphie_the_Buffalo) is studying to become a civil engineer. That means she has to know everything about concrete, including what ingredients go in it and how they affect the concrete's properties. She's currently writting up a project about concrete flow, and has asked you to help her figure out which ingredients are the most important. Let's use our new model selection techniques to help Ralphie out!

Data Source: Yeh, I-Cheng, "Modeling slump flow of concrete using second-order regressions and 
artificial neural networks," Cement and Concrete Composites, Vol.29, No. 6, 474-480, 
2007.

In [2]:
concrete.data = read.csv("Concrete.data")

concrete.data = concrete.data[, c(-1, -9, -11)]
names(concrete.data) = c("cement", "slag", "ash", "water", "sp", "course.agg", "fine.agg", "flow")

head(concrete.data)

,cement,slag,ash,water,sp,course.agg,fine.agg,flow
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,273,82,105,210,9,904,680,62.0
2,163,149,191,180,12,843,746,20.0
3,162,148,191,179,16,840,743,20.0
4,162,148,190,179,19,838,741,21.5
5,154,112,144,220,10,923,658,64.0
6,147,89,115,202,9,860,829,55.0


### 1. (a) Initial Inspections

Sometimes, the best way to start is to just jump in and mess around with the model. So let's do that. Create a linear model with `flow` as the response and all other columns as predictors.

Just by looking at the summary for your model, is there reason to believe that our model could be simpler?

In [3]:
# Your Code Here
concrete.lm = lm(data = concrete.data, formula = flow ~ .)
summary(concrete.lm)


Call:
lm(formula = flow ~ ., data = concrete.data)

Residuals:
    Min      1Q  Median      3Q     Max 
-30.880 -10.428   1.815   9.601  22.953 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)  
(Intercept) -252.87467  350.06649  -0.722   0.4718  
cement         0.05364    0.11236   0.477   0.6342  
slag          -0.00569    0.15638  -0.036   0.9710  
ash            0.06115    0.11402   0.536   0.5930  
water          0.73180    0.35282   2.074   0.0408 *
sp             0.29833    0.66263   0.450   0.6536  
course.agg     0.07366    0.13510   0.545   0.5869  
fine.agg       0.09402    0.14191   0.663   0.5092  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 12.84 on 95 degrees of freedom
Multiple R-squared:  0.5022,	Adjusted R-squared:  0.4656 
F-statistic: 13.69 on 7 and 95 DF,  p-value: 3.915e-12


Looking at the model summary shows that only one of the predictors has a p-value less than 0.05. Essentially, most of the predictors are not significant and so the model can be simpler if some are removed. It is likely also the case that a few of the predictors are having undesirable effects on the model as a whole. Figuring out which ones are best to remove is certainly a worthwhile task to make the model simpler and better.

### 1. (b) Backwards Selection
Our model has $7$ predictors. That is not too many, so we can use backwards selection to narrow them down to the most impactful.

Perform backwards selection on your model. You don't have to automate the backwards selection process.

In [4]:
# Your Code Here
library(tidyverse)
library(caret)
library(leaps)


set.seed(91)
train.control <- trainControl(method = "cv", number = 10)
step.model <- train(flow ~., data = concrete.data,
                    method = "leapBackward", 
                    tuneGrid = data.frame(nvmax = 1:7),
                    trControl = train.control
                    )

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [5]:
step.model$results

,nvmax,RMSE,Rsquared,MAE,RMSESD,RsquaredSD,MAESD
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,13.47424,0.4558112,11.53428,2.355786,0.2080745,2.222686
2,2,13.56651,0.4444407,11.48684,2.526269,0.2245271,2.389611
3,3,13.47827,0.4561530,11.53407,2.712882,0.2436323,2.657156
4,4,13.10484,0.4722376,11.08751,2.758823,0.2226460,2.760766
5,5,12.71436,0.5139511,10.88454,2.959819,0.2678322,3.126598
6,6,12.83089,0.5070470,11.00584,2.923395,0.2699133,3.117087
7,7,12.84088,0.5069901,11.03840,2.919280,0.2699540,3.106442


In [6]:
step.model$bestTune

,nvmax
,<int>
5,5


In [7]:
summary(step.model$finalModel)

Subset selection object
7 Variables  (and intercept)
           Forced in Forced out
cement         FALSE      FALSE
slag           FALSE      FALSE
ash            FALSE      FALSE
water          FALSE      FALSE
sp             FALSE      FALSE
course.agg     FALSE      FALSE
fine.agg       FALSE      FALSE
1 subsets of each size up to 5
Selection Algorithm: backward
         cement slag ash water sp  course.agg fine.agg
1  ( 1 ) " "    " "  " " "*"   " " " "        " "     
2  ( 1 ) " "    " "  " " "*"   " " " "        "*"     
3  ( 1 ) " "    " "  " " "*"   " " "*"        "*"     
4  ( 1 ) " "    " "  "*" "*"   " " "*"        "*"     
5  ( 1 ) "*"    " "  "*" "*"   " " "*"        "*"     

In [8]:
backward.select.lm = lm(data = concrete.data, formula = flow ~ cement + ash + water + course.agg + fine.agg)
summary(backward.select.m)

ERROR: Error in summary(backward.select.m): object 'backward.select.m' not found


### 1. (c) Objection!

Stop right there! Think about what you just did. You just removed the "worst" features from your model. But we know that a model will become less powerful when we remove features so we should check that it's still just as powerful as the original model. Use a test to check whether the model at the end of backward selection is significantly different than the model with all the features.

Describe why we want to balance explanatory power with simplicity.

In [ ]:
# Your Code Here
sprintf("The adjusted R-squared for the backward selection model is: %s", 
        summary(backward.select.lm)$adj.r.squared)
sprintf("The adjusted R-squared for the original model is: %s",
        summary(concrete.lm)$adj.r.squared)

Comparing the adjusted $R^2$ value for the original model and the backward selection model shows that the model contrived from backward selection of predictors explains the data slightly better, and most importantly not any worse, than the original model with all of the predictors. The values are above for comparison. We want to maximize the explainability of the data when we build models so that we can verify the relationship in the data via the slopes, which are more robust when $R^2$ is higher. Additionally, by balancing explanatory power with simplicity, we avoid over and under fitting of the model.

### 1. (d) Checking our Model

Ralphie is nervous about her project and wants to make sure our model is correct. She's found a function called `regsubsets()` in the leaps package which allows us to see which subsets of arguments produce the best combinations. Ralphie wrote up the code for you and the documentation for the function can be found [here](https://www.rdocumentation.org/packages/leaps/versions/2.1-1/topics/regsubsets). For each of the subsets of features, calculate the AIC, BIC and adjusted $R^2$. Plot the results of each criterion, with the score on the y-axis and the number of features on the x-axis. 

Do all of the criterion agree on how many features make the best model? Explain why the criterion will or will not always agree on the best model.

**Hint**: It may help to look at the attributes stored within the regsubsets summary using `names(rs)`.

In [ ]:
reg = regsubsets(flow ~ cement+slag+ash+water+sp+course.agg+fine.agg+flow, data=concrete.data, nvmax=6)
rs = summary(reg)
#rs$which

# Your Code Here

rs$outmat

n = length(concrete.data$flow)
p = apply(rs$which, 1, sum)
aic = rs$bic - log(n) * p + 2 * p

res = data.frame(x=1:6, bic=rs$bic, adjr2=rs$adjr2, aic=aic)

plot(x=1:6,y=rs$bic,xlab="Number of Features",ylab="BIC",main="regsubsets results (bic)")
plot(x=1:6,y=rs$adjr2,xlab="Number of Features",ylab="Adjusted R-squared",main="regsubsets results (adjr2)")
plot(x=1:6,y=aic,xlab="Number of Features",ylab="AIC",main="regsubsets results (aic)")

All of the criterion agree that 2 features makes for the best model. The lowest AIC/BIC values come from models with 2 features, and the hights adjusted R-squred comes from the model with 2 features. These values are all in agreeance. It is not always the case that these criteria will always agree since they all measure something different. R-squared quantifies how well the model explains observed data, AIC quantifies how well the model is suited for predicting future values, and BIC is much like AIC but better used in situations where explanation is the goal of the model since it penalizes the addition of more features.